# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
weather_df = pd.read_csv('../output_data/weather_DF.csv')


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Wind Speed,Max Temp
0,newport,100,GB,1590102937,79,51.59,-3.00,8.72,64.00
1,hobart,0,AU,1590102786,80,-42.88,147.33,4.70,51.01
2,puerto ayora,93,ES,1590103042,88,36.59,-6.23,2.24,71.01
3,tiksi,100,RU,1590103042,96,71.69,128.87,5.55,31.66
4,bluff,70,NZ,1590103042,79,-46.60,168.33,21.85,54.14


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
location = weather_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_df["Humidity"]

In [8]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
ideal_weather_df = weather_df.loc[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & \
                                   (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)].dropna()

ideal_weather_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Wind Speed,Max Temp
160,ilhabela,0,BR,1590103055,78,-23.78,-45.36,5.73,70.56
175,iturama,0,BR,1590103056,79,-19.73,-50.20,5.32,70.39
210,namibe,0,AO,1590103058,56,-15.20,12.15,1.57,75.96
222,boffa,0,GN,1590103059,77,10.17,-14.03,3.36,77.81
234,angoche,0,MZ,1590103059,79,-16.23,39.91,7.07,71.02
295,baghdad,0,IQ,1590103066,39,33.34,44.40,3.36,78.80
364,benguela,0,AO,1590103071,80,-12.58,13.41,2.33,74.64
373,buraydah,0,SA,1590103072,24,26.33,43.98,4.70,78.80
454,paranaiba,0,BR,1590103077,80,-19.68,-51.19,5.12,71.19


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = ideal_weather_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
160,ilhabela,BR,-23.78,-45.36,
175,iturama,BR,-19.73,-50.20,
210,namibe,AO,-15.20,12.15,
222,boffa,GN,10.17,-14.03,
234,angoche,MZ,-16.23,39.91,
295,baghdad,IQ,33.34,44.40,
364,benguela,AO,-12.58,13.41,
373,buraydah,SA,26.33,43.98,
454,paranaiba,BR,-19.68,-51.19,


In [17]:
gmap_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(gmap_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing... skipping.")
        
    print("------------") 

# Print end of search once searching is completed
print("-------DONE-------")


Retrieving Results for Index 160: ilhabela.
Closest hotel in ilhabela is Ilha Flat Hotel.
------------
Retrieving Results for Index 175: iturama.
Closest hotel in iturama is Avenida Park Hotel - Iturama - MG.
------------
Retrieving Results for Index 210: namibe.
Closest hotel in namibe is Hotel ibis Styles Iu Namibe.
------------
Retrieving Results for Index 222: boffa.
Closest hotel in boffa is Hôtel Niara Belly.
------------
Retrieving Results for Index 234: angoche.
Closest hotel in angoche is Guest House HESADA.
------------
Retrieving Results for Index 295: baghdad.
Closest hotel in baghdad is Baghdad Hotel.
------------
Retrieving Results for Index 364: benguela.
Closest hotel in benguela is iu Hotel Benguela.
------------
Retrieving Results for Index 373: buraydah.
Closest hotel in buraydah is Ewaa Express Hotel - Buraydah.
------------
Retrieving Results for Index 454: paranaiba.
Closest hotel in paranaiba is Cast Comfort Hotel.
------------
-------End of Search-------


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig